![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
# QuantConnect Research Notebook / Local Jupyter 环境

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from AlgorithmImports import *

# 初始化 QuantBook，用于获取价格历史
qb = QuantBook()

# 待分析股票列表
symbols = ["NVDA", "TSLA", "AVGO", "AXON", "TPL", "NFLX", "URI", "TDG", "BLDR", "FICO"]
api_key = "20dc406b88f1c6fd96020125eebd06f8:d3776cba675ad370af7abbb379351675"  # GuruFocus API key

In [ ]:
def fetch_gurufocus_data(ticker, modules):
    """调用 GuruFocus API 返回多个模块的财务数据"""
    base = f"https://api.gurufocus.com/public/user/{api_key}/stock/{ticker}/"
    data = {}
    for module in modules:
        url = base + module
        resp = requests.get(url)
        resp.raise_for_status()
        data[module] = resp.json()
    return data

def parse_fcf_from_cashflow(cashflow_json):
    """从现金流模块中提取年度自由现金流（返回 DataFrame）"""
    fcf = []
    for date, values in cashflow_json['financials']['cashflow_statement'].items():
        if 'FreeCashFlow' in values:
            fcf.append((pd.to_datetime(date), values['FreeCashFlow']))
    df = pd.DataFrame(fcf, columns=['Date', 'FreeCashFlow']).set_index('Date').sort_index()
    return df

def parse_ratio(ratios_json, field):
    """从 ratios 模块提取某个比率的历史值"""
    records = ratios_json['ratios']
    out = []
    for date, values in records.items():
        if field in values:
            out.append((pd.to_datetime(date), values[field]))
    df = pd.DataFrame(out, columns=['Date', field]).set_index('Date').sort_index()
    return df

def parse_valuation(valuation_json, field):
    """从 valuation 模块提取估值倍数"""
    records = valuation_json['valuation']
    out = []
    for date, values in records.items():
        if field in values:
            out.append((pd.to_datetime(date), values[field]))
    return pd.DataFrame(out, columns=['Date', field]).set_index('Date').sort_index()

# 存放分析结果
results = []



In [ ]:
for ticker in symbols:
    # 1. 获取价格历史（月度）
    symbol_obj = qb.add_equity(ticker, Resolution.Daily).symbol
    price_history = qb.history(symbol_obj, pd.to_datetime("2010-01-01"), pd.to_datetime("2025-08-01"), Resolution.Daily)['close']
    monthly_price = price_history.resample('M').last()
    
    # 2. 调用 GuruFocus API
    modules = ["cashflow", "ratios", "valuation"]
    data = fetch_gurufocus_data(ticker, modules)
    
    # 3. 解析所需字段
    fcf_df = parse_fcf_from_cashflow(data['cashflow'])        # 自由现金流
    roic_df = parse_ratio(data['ratios'], 'ROIC')            # ROIC
    revenue_df = parse_ratio(data['ratios'], 'Revenue')      # Revenue (示例)
    pfcf_df = parse_valuation(data['valuation'], 'PFCFRatio')# P/FCF
    
    # 4. 将财务数据按月前向填充，并对齐价格
    combined = pd.DataFrame(index=monthly_price.index)
    for df, name in [(fcf_df, 'FCF'), (roic_df, 'ROIC'), (revenue_df, 'Revenue'), (pfcf_df, 'PFCF')]:
        combined[name] = df['FreeCashFlow' if name=='FCF' else name].resample('M').ffill()
    combined['Price'] = monthly_price
    
    # 5. 计算增长率与简单因子得分（示例）
    fcf_growth = combined['FCF'].pct_change(4)                # 同比增长率 (按年)
    revenue_growth = combined['Revenue'].pct_change(4)
    roic = combined['ROIC']
    pfcf = combined['PFCF']
    # Z-score 标准化
    def zscore(s): return (s - s.mean())/s.std()
    score = zscore(fcf_growth) + zscore(revenue_growth) + zscore(roic) - zscore(pfcf)
    
    # 6. 保存结果
    results.append({
        'Ticker': ticker,
        'Latest_FCF': combined['FCF'].iloc[-1],
        'Latest_ROIC': roic.iloc[-1],
        'Latest_RevenueGrowth': revenue_growth.iloc[-1],
        'Latest_PFCF': pfcf.iloc[-1],
        'CompositeScore': score.iloc[-1]
    })
    
    # 7. 作图：价格 vs FCF（双轴）
    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax2 = ax1.twinx()
    ax1.plot(combined.index, combined['Price'], marker='o', label=f"{ticker} Price")
    ax2.plot(combined.index, combined['FCF']/1e9, linestyle='--', marker='s', label=f"{ticker} FCF (B)")
    ax1.set_title(f"{ticker} Price vs Free Cash Flow")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Price (USD)")
    ax2.set_ylabel("Free Cash Flow (Billions USD)")
    # 合并图例
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.tight_layout()
    plt.show()




In [ ]:
for ticker in symbols:
    # 1. 获取价格历史（月度）
    symbol_obj = qb.add_equity(ticker, Resolution.Daily).symbol
    history = qb.history(symbol_obj, pd.to_datetime("2010-01-01"), pd.to_datetime("2025-08-01"), Resolution.Daily)

    # 将 MultiIndex DataFrame 转换为以时间为索引的 Series
    price_df = history.reset_index().set_index('time')
    monthly_price = price_df['close'].resample('M').last()

    # 2. 调用 GuruFocus API
    modules = ["cashflow", "ratios", "valuation"]
    data = fetch_gurufocus_data(ticker, modules)
    
    # 3. 解析所需字段
    fcf_df = parse_fcf_from_cashflow(data['cashflow'])        # 自由现金流
    roic_df = parse_ratio(data['ratios'], 'ROIC')             # ROIC
    revenue_df = parse_ratio(data['ratios'], 'Revenue')       # Revenue (示例)
    pfcf_df = parse_valuation(data['valuation'], 'PFCFRatio') # P/FCF
    
    # 4. 将财务数据按月前向填充，并对齐价格
    combined = pd.DataFrame(index=monthly_price.index)
    for df, name in [(fcf_df, 'FCF'), (roic_df, 'ROIC'), (revenue_df, 'Revenue'), (pfcf_df, 'PFCF')]:
        col_name = 'FreeCashFlow' if name == 'FCF' else name
        df = df[[col_name]]
        df = df.rename(columns={col_name: name})
        combined[name] = df.resample('M').ffill()[name]
    combined['Price'] = monthly_price
    
    # 5. 计算增长率与简单因子得分（示例）
    fcf_growth = combined['FCF'].pct_change(4)                # 同比增长率 (按年)
    revenue_growth = combined['Revenue'].pct_change(4)
    roic = combined['ROIC']
    pfcf = combined['PFCF']
    
    def zscore(s): return (s - s.mean()) / s.std()
    score = zscore(fcf_growth) + zscore(revenue_growth) + zscore(roic) - zscore(pfcf)
    
    # 6. 保存结果
    results.append({
        'Ticker': ticker,
        'Latest_FCF': combined['FCF'].iloc[-1],
        'Latest_ROIC': roic.iloc[-1],
        'Latest_RevenueGrowth': revenue_growth.iloc[-1],
        'Latest_PFCF': pfcf.iloc[-1],
        'CompositeScore': score.iloc[-1]
    })
    
    # 7. 作图：价格 vs FCF（双轴）
    fig, ax1 = plt.subplots(figsize=(8, 4))
    ax2 = ax1.twinx()
    ax1.plot(combined.index, combined['Price'], marker='o', label=f"{ticker} Price")
    ax2.plot(combined.index, combined['FCF'] / 1e9, linestyle='--', marker='s', label=f"{ticker} FCF (B)")
    ax1.set_title(f"{ticker} Price vs Free Cash Flow")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Price (USD)")
    ax2.set_ylabel("Free Cash Flow (Billions USD)")

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
# 8. 汇总评分结果
results_df = pd.DataFrame(results).set_index('Ticker').sort_values('CompositeScore', ascending=False)
print(results_df)

In [8]:
from AlgorithmImports import *
import pandas as pd
import matplotlib.pyplot as plt

qb = QuantBook()

# 设置分析标的
tickers = ["NVDA", "TSLA", "AVGO", "NFLX", "FICO"]
start = datetime(2010, 1, 1)
end = datetime(2025, 8, 1)

fcf_dict = {}

for ticker in tickers:
    # 添加股票
    symbol = qb.add_equity(ticker).symbol
    
    # 获取 FineFundamental 历史数据
    fundamentals = qb.get_fundamental(symbol, start, end)
    
    # 提取自由现金流（按报告时间）
    rows = []
    for row in fundamentals:
        report_date = row.end_time
        fcf = row.financial_statements.cash_flow_statement.free_cash_flow
        if fcf is not None:
            rows.append((report_date, fcf))
    
    # 整理为 DataFrame
    df = pd.DataFrame(rows, columns=["Date", ticker]).set_index("Date")
    df = df[~df.index.duplicated(keep='last')]  # 去重
    fcf_dict[ticker] = df[ticker]

# 合并所有股票 FCF
fcf_df = pd.concat(fcf_dict.values(), axis=1)
fcf_df.columns = fcf_dict.keys()

# 绘图
plt.figure(figsize=(12, 6))
for ticker in fcf_df.columns:
    plt.plot(fcf_df.index, fcf_df[ticker]/1e9, label=ticker, marker='o')
plt.title("自由现金流（Free Cash Flow）历史趋势")
plt.ylabel("FCF（十亿美元）")
plt.xlabel("年份")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
